<a href="https://colab.research.google.com/github/nikita-ramesh/PDIOTApp/blob/main/jupyterNotebooks/PDIoT_lab3_social_signals_no_gyro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3 - Activity Recognition with Machine Learning**

This notebook implements a machine learning workflow to recognize different physical activities from Respeck sensor data. The dataset includes multiple 30-second recordings of various physical activities (e.g., ascending stairs, shuffle walking, sitting-standing) stored in separate CSV files for each activity.

You will then use the model you develop here and deploy it inside your Android app for live classification.

In this week, you will not have access to the full dataset as of yet. However, you can complete this lab by combining the data that you and your group mates have collected in Coursework 1 as proof-of-concept first for when you eventually receive the full dataset.


# Imports

In [1]:
import sklearn

In [2]:
# Importing libraries that will be used
import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

# Reading Files
Reading files from your dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Put in the path of your dataset here
your_dataset_path = "C:/Users/luise/OneDrive - University of Edinburgh/uni/pdiot/cw3 data/Respeck/Respiratory/"
your_dataset_path = "C:/Users/luise/OneDrive - University of Edinburgh/uni/pdiot/cw3 data/Respeck all/Respiratory/"
your_dataset_path = "/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/"

This line uses the glob module to find all file paths that match a specified pattern. The 'glob.glob()' function returns a list of file paths that match the given pattern. `your_dataset_path` should be the directory where your dataset files are located.

The `*` is a wildcard character that matches any string of characters,  so this pattern retrieves all folders in the 'your_dataset_path' directory.

Below is just an example of what your dataset folder can look like. You should refer to the Coursework 3 instructions on what classes your model(s) are expected to be able to classify. Within your dataset directory, there should be subfolders, each representing a class of activity.

In [5]:
glob.glob(your_dataset_path + "*")

['/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/breathingNormally',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/hyperventilation',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/other',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/coughing']

To see the files in each subfolder you can similarly do:

In [6]:
activity_folder = "other"
glob.glob(your_dataset_path + "/"+activity_folder+"/*")

['/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingRight_talking.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_sitting_eating.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingLeft_laughing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingStomach_talking.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingStomach_singing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingLeft_singing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingBack_talking.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_standing_singing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/R

# Functions

## Load list of files in an activity folder

In [7]:
def load_files_from_folder(folder_path):
    """
    Load all CSV files from a folder and return a list of file paths.

    Parameters:
    folder_path (str): The path to the folder containing CSV files.

    Returns:
    list: A list of file paths for all CSV files in the folder.
    """

    # Initialize an empty list to store the full file paths of the CSV files
    file_paths = []

    # Loop through all the files in the given folder
    for file_name in os.listdir(folder_path):
        # Check if the file has a .csv extension (ignores other files)
        if file_name.endswith('.csv'):
            # Construct the full file path by joining the folder path and the file name
            full_file_path = os.path.join(folder_path, file_name)

            # Append the full file path to the file_paths list
            file_paths.append(full_file_path)

    # Return the complete list of CSV file paths
    return file_paths

## Train and test set split from list of files

In [8]:
def split_files(file_list, test_size=0.2):
    """
    Split the list of files into training and test sets.

    Parameters:
    file_list (list): List of file paths to be split into train and test sets.
    test_size (float): The proportion of files to allocate to the test set.
                       Default is 0.2, meaning 20% of the files will be used for testing.

    Returns:
    tuple:
        - train_files (list): List of file paths for the training set.
        - test_files (list): List of file paths for the test set.
    """

    # Split the file list into training and test sets using train_test_split from scikit-learn
    # test_size defines the proportion of the data to use as the test set (default is 20%)
    # shuffle=True ensures that the files are shuffled randomly before splitting
    train_files, test_files = train_test_split(file_list, test_size=test_size, shuffle=True)

    # Return the train and test file lists
    return train_files, test_files

## Sliding Window

In time series Activity Recognition, a sliding window is a commonly used technique to segment continuous sensor data (such as accelerometer readings) into smaller, fixed-length overlapping or non-overlapping time intervals, or windows. Each window contains a sequence of sensor measurements that represent a short period of time, and this segmented data is used to extract features or make predictions about the activity happening within that window.

### Key Concepts of a Sliding Window
1.   **Window Size:** This refers to the length of each segment or window, typically defined in terms of the number of time steps or the duration (e.g., 2 seconds). The window size should be chosen carefully to capture enough information about the activity without making the window too large.
2.   **Step Size:** The step size determines how far the window moves forward after each step. If the step size is smaller than the window size, the windows will overlap. For example, if the window size is 5 seconds and the step size is 2 seconds, there will be a 3-second overlap between consecutive windows. Overlapping windows provide more data for analysis and can help smooth out predictions by capturing transitional activities.
3.   **Non-Overlapping Windows:** If the step size is equal to the window size, the windows do not overlap. This method provides distinct segments of data but may miss transitional phases between activities.

### Why Sliding Windows for Activity Recognition?

* Segmentation of Continuous Data: Activity recognition systems work with continuous streams of sensor data, and the sliding window helps segment these into manageable pieces to classify activities within specific intervals.

* Context Capturing: Human activities are often complex and spread across time. By using a sliding window, you can capture context across a short duration, which may include transitions or small fluctuations in the activity (e.g., a person moving from sitting to standing).

* Feature Extraction: Within each window, features such as mean, variance, frequency domain features, etc., can be extracted to help classify the activity.

* Real-Time Recognition: In real-time systems, the sliding window allows for continuous monitoring and updating of predictions as new data arrives.



In [9]:
def load_and_apply_sliding_windows(file_paths, window_size, step_size, label):
    """
    Load the data from each file, apply sliding windows, and return the windows and labels.

    Parameters:
    file_paths (list): List of file paths to CSV files. Each file contains sensor data (e.g., accelerometer, gyroscope).
    window_size (int): The size of each sliding window (number of time steps).
    step_size (int): The step size (stride) between consecutive windows.
    label (int or str): The label for the activity corresponding to the folder.
                        This label will be assigned to each sliding window extracted from the data.

    Returns:
    tuple:
        - windows (numpy.ndarray): A 3D array of sliding windows, where each window has the shape
                                   (num_windows, window_size, num_features).
        - labels (numpy.ndarray): A 1D array of labels, where each label corresponds to a sliding window.
    """
    # Initialize lists to store sliding windows and their corresponding labels
    windows = []
    labels = []

    # Loop through each file in the provided file paths
    for file_path in file_paths:
        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file_path)

        # Select the columns containing the necessary sensor data (acceleration and gyroscope readings)
        # These columns might vary depending on your dataset's structure
        data = data[['accel_x', 'accel_y', 'accel_z']]

        # Convert the DataFrame into a numpy array for faster processing in the sliding window operation
        data = data.to_numpy()

        # Get the number of samples (rows) and features (columns) in the data
        num_samples, num_features = data.shape

        # Apply sliding windows to the data
        # The range function defines the start of each window, moving step_size increments at a time
        for i in range(0, num_samples - window_size + 1, step_size):
            # Extract a window of size 'window_size' from the current position 'i'
            window = data[i:i + window_size, :]

            # Append the window to the windows list
            windows.append(window)

            # Assign the activity label to the window and append it to the labels list
            labels.append(label)

    # Convert the lists of windows and labels into numpy arrays for efficient numerical operations
    return np.array(windows), np.array(labels)

## Load and Split Train Test for Each Activity Folder

This function processes the sensor data for a specific activity, such as 'walking' or 'running', stored in its respective folder. It splits the data into training and testing sets, applies sliding windows, and labels the windows with the corresponding activity. This function can be used repeatedly for each activity to process and prepare data for training and evaluation.

In [10]:
def process_activity(activity, label, dataset_path, window_size=50, step_size=50, test_size=0.2):
    """
    Processes an activity folder by loading the file list, splitting them into
    train and test sets, and applying sliding windows to the files.

    Args:
        activity (str): Name of the activity (folder name). This refers to the specific physical activity
                        like 'walking', 'running', etc.
        label (int): Numeric label corresponding to the activity, used for classification.
        dataset_path (str): Base path where the activity folders are located.
        window_size (int): Size of the sliding window, i.e., the number of time steps included in each window.
                           Default is 50.
        step_size (int): Step size for the sliding window, i.e., how far the window moves along the data.
                         Default is 50 (no overlap between windows).
        test_size (float): Proportion of files to use for testing. Default is 0.2, meaning 20% of files will
                           be allocated to the test set.

    Returns:
        tuple:
            - train_windows (numpy.ndarray): Sliding windows from the training files.
            - train_labels (numpy.ndarray): Corresponding labels for the training windows.
            - test_windows (numpy.ndarray): Sliding windows from the test files.
            - test_labels (numpy.ndarray): Corresponding labels for the test windows.
    """
    # Construct the full folder path where the activity files are stored
    folder_path = os.path.join(dataset_path, activity)

    # Load all CSV file paths for the given activity from the folder
    file_list = load_files_from_folder(folder_path)

    # Split the file list into training and testing sets
    # train_files: files used for training
    # test_files: files used for testing
    train_files, test_files = split_files(file_list, test_size=test_size)

    # Apply sliding windows to the training files
    # The function 'load_and_apply_sliding_windows' returns the sliding windows (segments) and their corresponding labels
    train_windows, train_labels = load_and_apply_sliding_windows(train_files, window_size, step_size, label)

    # Apply sliding windows to the testing files
    test_windows, test_labels = load_and_apply_sliding_windows(test_files, window_size, step_size, label)

    # Return the sliding windows and their labels for both training and testing sets
    return train_windows, train_labels, test_windows, test_labels

## Combine Data
The function combines the sliding window data and their corresponding labels from multiple activities (e.g., walking, running, etc.) into single arrays.

In [11]:
def combine_data(train_test_data, data_type):
    """
    Combines the sliding windows and labels from all activities into a single
    array for either training or testing.

    Args:
        train_test_data (dict): Dictionary containing the sliding window data for all activities.
                                Each key in the dictionary corresponds to an activity, and the value is another
                                dictionary with the keys 'train_windows', 'train_labels', 'test_windows', 'test_labels'.
        data_type (str): Either 'train' or 'test' to specify which data to combine (e.g., 'train_windows' or 'test_windows').

    Returns:
        tuple:
            - windows (numpy.ndarray): Concatenated windows from all activities for either training or testing.
            - labels (numpy.ndarray): Concatenated labels corresponding to the windows from all activities.
    """

    # Extract the list of sliding windows for the specified data type (either 'train' or 'test') from each activity
    # For example, if data_type is 'train', it extracts 'train_windows' for all activities
    windows_list = [train_test_data[activity][f'{data_type}_windows'] for activity in train_test_data]

    # Similarly, extract the list of labels corresponding to the windows for each activity
    labels_list = [train_test_data[activity][f'{data_type}_labels'] for activity in train_test_data]

    # Concatenate all the sliding windows into a single numpy array along the first axis (rows)
    # This creates one large array of windows from all the activities combined
    concatenated_windows = np.concatenate(windows_list, axis=0)

    # Concatenate all the labels into a single numpy array along the first axis (rows)
    # The labels are now aligned with the concatenated windows
    concatenated_labels = np.concatenate(labels_list, axis=0)

    # Return the concatenated windows and labels as a tuple
    return concatenated_windows, concatenated_labels

## 1D CNN Model

This function, `build_1d_cnn_model`, creates and compiles a 1D Convolutional Neural Network (CNN) for multi-class classification tasks.

### Function Overview

Input Parameters
* `input_shape`: Specifies the shape of the input data. It represents (timesteps, features), where timesteps refer to the length of the time series (e.g., 50 windows), and features represent the number of measurements in each time step (e.g., accelerometer readings).
* `num_classes`: The number of output classes for the classification problem. For example, if you're classifying six different activities, num_classes would be 6.

Returns
* The function returns a compiled 1D CNN model that is ready to be trained on your data.

<hr>

### Function Breakdown
1.   Model Initialization:
    * `model = Sequential()`: Initializes a Sequential model, which means layers will be stacked on top of each other in a linear fashion.
2.   First Convolutional Layer
    * `Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape)`
        * This is the first 1D convolutional layer
        * `filters=64`: The layer applies 64 filters (or kernels) over the input data.
        * `kernel_size=3`: Each filter will cover 3 timesteps at a time (a window of 3).
        * `activation='relu'`: The Rectified Linear Unit (ReLU) activation function introduces non-linearity and helps the model learn complex patterns.
        * `input_shape=input_shape`: Specifies the shape of the input data.
    * `MaxPooling1D(pool_size=2)`: This pooling layer reduces the dimensionality of the data by taking the maximum value from each 2-timestep window (`pool_size=2`). This helps reduce computational complexity and captures the most important features.
3. Second Convolutional Layer:
    * `Conv1D(filters=128, kernel_size=3, activation='relu')`
        * This is the second convolutional layer, similar to the first, but with 128 filters, which allow the network to learn more complex features from the data.
        * `kernel_size=3` and activation='relu' function in the same way as the first Conv1D layer.
    * `MaxPooling1D(pool_size=2)`: Another pooling layer to downsample the output, further reducing the data’s dimensionality.
4. Flattening Layer:
    * `Flattening`: Converts the 2D output of the convolutional and pooling layers into a 1D vector. This is necessary because the next layer is fully connected, and it requires a 1D input.
5. Fully Connected Layer:
    * `Dense(128, activation='relu')`: This is a fully connected layer with 128 units/neurons. Each neuron is connected to every input from the flattened output. The ReLU activation function is used again to introduce non-linearity and help the model learn complex relationships.
6. Dropout Layer:
    * `Dropout(0.5)`: This layer randomly sets 50% of the neurons to zero during training to prevent overfitting. It helps the model generalize better to unseen data.
7. Output Layer:
    * `Dense(num_classes, activation='softmax')`: This is the output layer with num_classes neurons, one for each class in the classification problem. The softmax activation function ensures the output values represent probabilities that sum to 1, useful for multi-class classification.
8. Compiling the model
    * model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']):
        * Optimizer: 'adam': Adam is an optimization algorithm that adjusts the learning rate during training to improve performance.
        * Loss: 'categorical_crossentropy': This loss function is used for multi-class classification problems where the target variable is one-hot encoded (i.e., represented as a vector of 0s and 1s).
        * Metrics: ['accuracy']: The accuracy metric is used to evaluate the model’s performance during training and testing.


In [12]:
def build_1d_cnn_model(input_shape, num_classes):
    """
    Builds and compiles a 1D CNN model for multi-class classification.

    Args:
        input_shape (tuple): The shape of the input data (timesteps, features).
        num_classes (int): The number of output classes.

    Returns:
        model (Sequential): Compiled 1D CNN model.
    """
    model = Sequential()

    # First Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # Second Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    # Flatten the output from the convolutional layers
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))

    # Dropout layer for regularization
    # You can try experimenting with different dropout rates
    model.add(Dropout(0.5))

    # Output layer with softmax for multi-class classification
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #  Prints a detailed summary of the model, showing the layers, their output shapes, and the number of trainable parameters
    model.summary()

    return model

# Classification Pipeline

## Step 1: Prepare and Preprocess the Data

In [13]:
# Define activity folders and corresponding labels
# Each key is the name of the physical activity, and the corresponding value is the numeric label
# These labels will be used as the target variable for classification.
activities = {
    'breathingNormally': 0,
    'coughing': 1,
    'hyperventilation': 2,
    'other': 3
}

In [14]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data[activity]['train_windows'], train_test_data[activity]['train_labels'],
     train_test_data[activity]['test_windows'], train_test_data[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=50, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.

Now that each activity has been processed and stored in train_test_data, we need to combine the sliding windows and labels from all activities into unified arrays (one for training and one for testing) for model training.

In [15]:
# Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train, y_train = combine_data(train_test_data, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test, y_test = combine_data(train_test_data, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.

### One-Hot Encode Labels (for multi-class classification)
If you have more than two classes, you'll need to one-hot encode the labels, especially if your model will use categorical cross-entropy loss.

One-Hot Encoding converts categorical labels into binary vectors (one-hot encoded format). Each class label is represented as a binary vector with 1 for the correct class and 0 for others. This is necessary for training models that use categorical_crossentropy as the loss function, such as a neural network.

In [16]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [17]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot shape: {y_train_one_hot.shape}, y_test_one_hot shape: {y_test_one_hot.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

X_train shape: (57240, 50, 3), y_train shape: (57240,)
X_test shape: (14327, 50, 3), y_test shape: (14327,)
y_train_one_hot shape: (57240, 4), y_test_one_hot shape: (14327, 4)


## Step 2: Build the 1D-CNN Model
Call our `build_1d_cnn_model` functionto build our model

In [18]:
# Determine the input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2])

# Determine the number of output classes (num_classes)
num_classes = y_train_one_hot.shape[1]

# Build and compile the model
# The function will return a compiled model ready for training
model = build_1d_cnn_model(input_shape, num_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 48, 64)              │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 24, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 22, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 11, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1408)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         180,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 206,212 (805.52 KB)

 Trainable params: 206,212 (805.52 KB)

 Non-trainable params: 0 (0.00 B)

## Step 3: Train the CNN Model

Train the 1D CNN model using the training data and validate on the test data. The model will learn to map input sliding windows to their corresponding activity labels.

`model.fit()` is used to train the neural network model. It takes several parameters:
* `X_train`: The input training data (sliding windows), with shape (num_samples, window_size, num_features).
* `y_train_one_hot`: The corresponding one-hot encoded labels for the training data, with shape (num_samples, num_classes).
* `epochs`: Number of times the entire training dataset is passed through the model. You can try adjusting the number of epochs and compare the difference in model performance. In this case, we are training for 20 epochs, meaning the model will see the entire training set 20 times.
* `batch_size`: Number of samples processed before the model's weights are updated. Here, the batch size is set to 32, meaning the model will process 32 samples at a time before updating its parameters.
* `validation_data`: This parameter allows us to evaluate the model's performance on the test data after each epoch.
*`(X_test, y_test_one_hot)`: These are the input test data and corresponding one-hot encoded test labels.

# Luise's accuracy improvement trials:

## ReduceLROnPlateau

In [ ]:
# 1. ReduceLROnPlateau // reduce learning rate when val_accuracy stops improving
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',   # Track validation accuracy for learning rate adjustments
    factor=0.5,               # Reduce learning rate by half when triggered
    patience=3,               # Wait for 3 epochs of no improvement before reducing
    min_lr=1e-5               # Set a minimum learning rate to prevent it from going too low
)
history = model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot),   # Validate on the test set after each epoch
                    callbacks=[lr_scheduler])

## 2. automatic hyperparameter tuner:

In [21]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.regularizers import l2
import keras_tuner as kt

In [23]:
def build_tuning_model(hp):
    model = Sequential()

    # First Conv1D Layer
    model.add(Conv1D(
        filters=hp.Choice('filters_1', values=[64, 128, 256]),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5, 7]),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=hp.Choice('dropout_1', values=[0.3, 0.5, 0.6])))

    # Second Conv1D Layer
    model.add(Conv1D(
        filters=hp.Choice('filters_2', values=[64, 128, 256]),
        kernel_size=hp.Choice('kernel_size_2', values=[3, 5, 7]),
        activation='relu',
        kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3]))
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=hp.Choice('dropout_2', values=[0.3, 0.5, 0.6])))

    # Optional LSTM Layer
    if hp.Boolean("use_lstm"):
        model.add(LSTM(units=hp.Choice('lstm_units', values=[32, 64, 128]), return_sequences=False))

    # Dense Layers
    model.add(Flatten())

    model.add(Dense(128, activation='relu')) #gpt suggested 64 but original model has 128

    model.add(Dropout(rate=hp.Choice('dropout_3', values=[0.3, 0.5, 0.6])))

    model.add(Dense(num_classes, activation='softmax'))  # Assuming 4 classes for respiratory activities

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    return model

In [ ]:
tuner = kt.RandomSearch(
    build_tuning_model,
    objective='val_accuracy',  # Tuning for validation accuracy
    max_trials=20,             # Number of different hyperparameter sets to try
    executions_per_trial=2,    # Average results over multiple runs to reduce variance
    directory='my_tuning_dir', # Directory to save tuning results
    project_name='respiratory_activity_classification_no_gyro'
)

# Split your data if not already done
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

tuner.search(X_train, y_train_one_hot,
             epochs=20,               # Number of epochs for each trial
             validation_data=(X_test, y_test_one_hot),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

best_model1 = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
best_model1.evaluate(X_test, y_test_one_hot)  # Replace with your test data
tuner.results_summary()

Trial 20 Complete [00h 02m 25s]
val_accuracy: 0.5933031141757965

Best val_accuracy So Far: 0.6573770344257355
Total elapsed time: 01h 20m 36s


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 48, 256)             │           2,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 24, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 128)             │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 20, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 226,308 (884.02 KB)

 Trainable params: 225,540 (881.02 KB)

 Non-trainable params: 768 (3.00 KB)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 32 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6717 - loss: 0.8577
Results summary
Results in my_tuning_dir/respiratory_activity_classification_no_gyro
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 16 summary
Hyperparameters:
filters_1: 256
kernel_size_1: 3
dropout_1: 0.3
filters_2: 128
kernel_size_2: 5
l2_regularization: 0.001
dropout_2: 0.3
use_lstm: True
dropout_3: 0.3
learning_rate: 0.001
lstm_units: 64
Score: 0.6573770344257355

Trial 11 summary
Hyperparameters:
filters_1: 256
kernel_size_1: 3
dropout_1: 0.3
filters_2: 128
kernel_size_2: 7
l2_regularization: 0.001
dropout_2: 0.5
use_lstm: True
dropout_3: 0.6
learning_rate: 0.001
lstm_units: 32
Score: 0.6270666122436523

Trial 05 summary
Hyperparameters:
filters_1: 256
kernel_size_1: 3
dropout_1: 0.5
filters_2: 128
kernel_size_2: 5
l2_regularization: 0.0001
dropout_2: 0.6
use_lstm: False
dropout_3: 0.6
learning_rate: 0.001
lstm_units: 128
Score: 0.6212417185306549

Trial 00 summary
Hyperparam

In [24]:
# best model tuning found:
def build_best_tuning_model(custom_input_shape=input_shape):
    model = Sequential()

    # First Conv1D Layer
    model.add(Conv1D(
        filters=128,
        kernel_size=5,
        activation='relu',
        input_shape=custom_input_shape
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=0.3))

    # Second Conv1D Layer
    model.add(Conv1D(
        filters=256,
        kernel_size=5,
        activation='relu',
        kernel_regularizer=l2(1e-3)
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=0.3))

    # Optional LSTM Layer
    model.add(LSTM(units=128, return_sequences=False))

    # Dense Layers
    model.add(Flatten())

    model.add(Dense(128, activation='relu')) #gpt suggested 64 but original model has 128

    model.add(Dropout(rate=0.3))

    model.add(Dense(num_classes, activation='softmax'))  # Assuming 4 classes for respiratory activities

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    return model

best_model = build_best_tuning_model()
history = best_model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot)   # Validate on the test set after each epoch
                )

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 46, 128)             │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 46, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 19, 256)             │         164,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 19, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 381,828 (1.46 MB)

 Trainable params: 381,060 (1.45 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.5097 - loss: 1.3190 - val_accuracy: 0.5461 - val_loss: 1.1594
Epoch 2/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.5568 - loss: 1.0899 - val_accuracy: 0.5698 - val_loss: 1.0377
Epoch 3/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.5781 - loss: 1.0117 - val_accuracy: 0.5714 - val_loss: 1.0118
Epoch 4/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.5958 - loss: 0.9699 - val_accuracy: 0.5951 - val_loss: 0.9767
Epoch 5/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6128 - loss: 0.9403 - val_accuracy: 0.6038 - val_loss: 1.0112
Epoch 6/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6252 - loss: 0.9197 - val_accuracy: 0.5972 - val_loss: 0.9437
Epoch 7/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6376 - loss: 0.8911 - val_accuracy: 0.6158 - val_loss: 0.9265
Epoch 8/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6386 - loss: 0

In [ ]:
# custom cell to rebuild modified versions of model
model = build_1d_cnn_model(input_shape, num_classes)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 48, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 22, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 11, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       180,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 206,788 (807.77 KB)

 Trainable params: 206,788 (807.77 KB)

 Non-trainable params: 0 (0.00 B)

## Step 4: Evaluate the Model
After training, you can evaluate the model on the test set:

In [ ]:
history = model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot)   # Validate on the test set after each epoch
                )

In [ ]:
# Get predicted probabilities for the test set
y_pred_probs = best_model.predict(X_test)

# Convert the predicted probabilities to class labels (taking the argmax of the probabilities)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Convert the true test labels from one-hot encoding back to class labels
y_true_classes = np.argmax(y_test_one_hot, axis=1)

# Generate the classification report
report = classification_report(y_true_classes, y_pred_classes, digits=4)

# Print the classification report
print(report)

448/448 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
              precision    recall  f1-score   support

           0     0.2351    0.1613    0.1914      2789
           1     0.1292    0.0231    0.0392      2161
           2     0.1492    0.8199    0.2525      2166
           3     0.7368    0.0117    0.0230      7200

    accuracy                         0.1649     14316
   macro avg     0.3126    0.2540    0.1265     14316
weighted avg     0.4585    0.1649    0.0930     14316



As you can see from the model performance results, the classification performance isn't exactly impressive. For Coursework 3, your group should explore and experiment with various models, parameters, and techniques in order to improve your model's performance.

# Exporting your model to TFLite

You can use the TFLiteConverter class provided by TensorFlow to convert your trained model into the TensorFlow Lite format. We export models to TensorFlow Lite (TFLite) for several reasons, primarily because TFLite is designed for deployment on edge devices, such as mobile phones, embedded systems, IoT devices, and microcontrollers, where computational resources and power are limited. This is necessary as you will be running your ML models on your Android devices to perform live classification.

In [ ]:
# Convert the trained Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # model is your trained Keras model
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully exported to model.tflite")

INFO:tensorflow:Assets written to: C:\Users\luise\AppData\Local\Temp\tmppf3rz10g\assets


INFO:tensorflow:Assets written to: C:\Users\luise\AppData\Local\Temp\tmppf3rz10g\assets


Saved artifact at 'C:\Users\luise\AppData\Local\Temp\tmppf3rz10g'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2847604399040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847604400976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847605255056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847605246432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847605248720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847614338848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847614349936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847614349232: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model successfully exported to model.tflite


# Good job!
This is the end of Lab 3. In the next lab, you will focus on deploying your machine learning model onto your Android App in order to classify activities in real-time.

# GPT's suggested models

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [ ]:
# Define parameters
input_shape = X_train.shape[1:]  # (window_size, num_features)
num_classes = y_train_one_hot.shape[1]  # 4 respiratory states

# Build the model
gpt_model1 = Sequential()

# Convolutional layers to capture spatial patterns
gpt_model1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
gpt_model1.add(MaxPooling1D(pool_size=2))
gpt_model1.add(Dropout(0.3))

gpt_model1.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
gpt_model1.add(MaxPooling1D(pool_size=2))
gpt_model1.add(Dropout(0.3))

# LSTM layer to capture temporal dependencies
gpt_model1.add(LSTM(100, return_sequences=True))
gpt_model1.add(LSTM(50))

# Fully connected layers for classification
gpt_model1.add(Dense(50, activation='relu'))
gpt_model1.add(Dropout(0.5))
gpt_model1.add(Dense(num_classes, activation='softmax'))

# Compile the model
gpt_model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
]

# Train the model
history = gpt_model1.fit(
    X_train, y_train_one_hot,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),
    callbacks=callbacks
)

# Evaluate the model on test data
test_loss, test_accuracy = gpt_model1.evaluate(X_test, y_test_one_hot)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Check class-wise accuracy
y_pred = gpt_model1.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_one_hot, axis=1)

report = classification_report(y_true_classes, y_pred_classes, target_names=['breathingNormally', 'coughing', 'hyperventilation', 'other'])
print(report)


Epoch 1/50
 527/1789 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.4937 - loss: 1.2657

KeyboardInterrupt: 

### model 2

In [ ]:
from tensorflow.keras.layers import Bidirectional
from sklearn.utils import class_weight
import numpy as np

In [ ]:
y_true_classes = np.argmax(y_train_one_hot, axis=1)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_true_classes), y=y_true_classes)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
# Update your model with deeper layers and bidirectional LSTM

gpt_model2 = Sequential()

# Convolutional layers
gpt_model2.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape))
gpt_model2.add(MaxPooling1D(pool_size=2))
gpt_model2.add(Dropout(0.4))

gpt_model2.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
gpt_model2.add(MaxPooling1D(pool_size=2))
gpt_model2.add(Dropout(0.4))

# Bidirectional LSTM layer
gpt_model2.add(Bidirectional(LSTM(150, return_sequences=True)))
gpt_model2.add(Bidirectional(LSTM(100)))

# Dense layers
gpt_model2.add(Dense(100, activation='relu'))
gpt_model2.add(Dropout(0.5))
gpt_model2.add(Dense(num_classes, activation='softmax'))

# Compile with a lower learning rate
gpt_model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train with class weights if needed
history = gpt_model2.fit(
    X_train, y_train_one_hot,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),
    callbacks=callbacks,
    class_weight=class_weights_dict  # Use class weights if classes are imbalanced
)


Epoch 1/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 55s 27ms/step - accuracy: 0.2924 - loss: 1.3370 - val_accuracy: 0.3323 - val_loss: 1.1616 - learning_rate: 5.0000e-04
Epoch 2/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 54s 30ms/step - accuracy: 0.3706 - loss: 1.0708 - val_accuracy: 0.3462 - val_loss: 1.0897 - learning_rate: 5.0000e-04
Epoch 3/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 60s 34ms/step - accuracy: 0.3929 - loss: 1.0103 - val_accuracy: 0.3759 - val_loss: 1.0771 - learning_rate: 5.0000e-04
Epoch 4/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 64s 36ms/step - accuracy: 0.4133 - loss: 0.9484 - val_accuracy: 0.4281 - val_loss: 1.0287 - learning_rate: 5.0000e-04
Epoch 5/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 60s 33ms/step - accuracy: 0.4315 - loss: 0.9324 - val_accuracy: 0.4045 - val_loss: 1.0603 - learning_rate: 5.0000e-04
Epoch 6/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 54s 30ms/step - accuracy: 0.4429 - loss: 0.9122 - val_accuracy: 0.4316 - val_loss: 0.9996 - learning_rate: 5.0000e-04
Epoch 7/100
1787/1787 ━━━━━━━━━━━━

best model with class balancing:

In [ ]:
# Train with class weights if needed
history = best_model.fit(
    X_train, y_train_one_hot,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),
    callbacks=callbacks,
    class_weight=class_weights_dict  # Use class weights if classes are imbalanced
)

Epoch 1/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.6119 - loss: 0.8103 - val_accuracy: 0.5926 - val_loss: 0.9284 - learning_rate: 0.0010
Epoch 2/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 38s 21ms/step - accuracy: 0.6113 - loss: 0.8017 - val_accuracy: 0.5695 - val_loss: 1.0352 - learning_rate: 0.0010
Epoch 3/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.5704 - loss: 0.8743 - val_accuracy: 0.5810 - val_loss: 0.9954 - learning_rate: 0.0010
Epoch 4/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.6149 - loss: 0.7899 - val_accuracy: 0.5532 - val_loss: 1.0148 - learning_rate: 0.0010
Epoch 5/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 36s 20ms/step - accuracy: 0.6019 - loss: 0.8026 - val_accuracy: 0.5896 - val_loss: 0.9696 - learning_rate: 0.0010
Epoch 6/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 37s 21ms/step - accuracy: 0.6153 - loss: 0.7833 - val_accuracy: 0.5847 - val_loss: 0.9815 - learning_rate: 0.0010
Epoch 7/100
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step -

### adding summary features

In [ ]:
import numpy as np
import scipy.stats as stats

In [ ]:
def load_and_apply_sliding_windows2(file_paths, window_size, step_size, label):
    """
    Load the data from each file, apply sliding windows, and return the windows and labels.

    Parameters:
    file_paths (list): List of file paths to CSV files. Each file contains sensor data (e.g., accelerometer, gyroscope).
    window_size (int): The size of each sliding window (number of time steps).
    step_size (int): The step size (stride) between consecutive windows.
    label (int or str): The label for the activity corresponding to the folder.
                        This label will be assigned to each sliding window extracted from the data.

    Returns:
    tuple:
        - windows (numpy.ndarray): A 3D array of sliding windows, where each window has the shape
                                   (num_windows, window_size, num_features).
        - labels (numpy.ndarray): A 1D array of labels, where each label corresponds to a sliding window.
    """
    # Initialize lists to store sliding windows and their corresponding labels
    windows = []
    labels = []

    # Loop through each file in the provided file paths
    for file_path in file_paths:
        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file_path)

        # Select the columns containing the necessary sensor data (acceleration and gyroscope readings)
        # These columns might vary depending on your dataset's structure
        data = data[['accel_x', 'accel_y', 'accel_z']]

        # Convert the DataFrame into a numpy array for faster processing in the sliding window operation
        data = data.to_numpy()

        # Get the number of samples (rows) and features (columns) in the data
        num_samples, num_features = data.shape

        # Apply sliding windows to the data
        # The range function defines the start of each window, moving step_size increments at a time
        for i in range(0, num_samples - window_size + 1, step_size):
            # Extract a window of size 'window_size' from the current position 'i'
            window = data[i:i + window_size, :]

            # Calculate statistics for each axis in the window
            mean = np.mean(window, axis=0)
            std = np.std(window, axis=0)
            min_val = np.min(window, axis=0)
            max_val = np.max(window, axis=0)
            skewness = stats.skew(window, axis=0)
            kurtosis = stats.kurtosis(window, axis=0)
            # Concatenate features into a single array
            window_features = np.concatenate([mean, std, min_val, max_val, skewness, kurtosis])

            # Reshape window_features to match the dimensions for concatenation
            # Expand dimensions to make it compatible with the window (1, num_statistical_features)
            window_features = np.expand_dims(window_features, axis=0)

            # Repeat window_features along the first dimension to match the window shape
            window_features_expanded = np.repeat(window_features, window.shape[0], axis=0)

            # Concatenate window_features with the original window along the last axis
            augmented_window = np.concatenate([window, window_features_expanded], axis=1)

            # Append the window to the windows list
            windows.append(augmented_window)

            # Assign the activity label to the window and append it to the labels list
            labels.append(label)


    # Convert the lists of windows and labels into numpy arrays for efficient numerical operations
    return np.array(windows), np.array(labels)

In [ ]:
def process_activity2(activity, label, dataset_path, window_size=50, step_size=50, test_size=0.2):
    """
    Processes an activity folder by loading the file list, splitting them into
    train and test sets, and applying sliding windows to the files.

    Args:
        activity (str): Name of the activity (folder name). This refers to the specific physical activity
                        like 'walking', 'running', etc.
        label (int): Numeric label corresponding to the activity, used for classification.
        dataset_path (str): Base path where the activity folders are located.
        window_size (int): Size of the sliding window, i.e., the number of time steps included in each window.
                           Default is 50.
        step_size (int): Step size for the sliding window, i.e., how far the window moves along the data.
                         Default is 50 (no overlap between windows).
        test_size (float): Proportion of files to use for testing. Default is 0.2, meaning 20% of files will
                           be allocated to the test set.

    Returns:
        tuple:
            - train_windows (numpy.ndarray): Sliding windows from the training files.
            - train_labels (numpy.ndarray): Corresponding labels for the training windows.
            - test_windows (numpy.ndarray): Sliding windows from the test files.
            - test_labels (numpy.ndarray): Corresponding labels for the test windows.
    """
    # Construct the full folder path where the activity files are stored
    folder_path = os.path.join(dataset_path, activity)

    # Load all CSV file paths for the given activity from the folder
    file_list = load_files_from_folder(folder_path)

    # Split the file list into training and testing sets
    # train_files: files used for training
    # test_files: files used for testing
    train_files, test_files = split_files(file_list, test_size=test_size)

    # Apply sliding windows to the training files
    # The function 'load_and_apply_sliding_windows' returns the sliding windows (segments) and their corresponding labels
    train_windows, train_labels = load_and_apply_sliding_windows2(train_files, window_size, step_size, label)

    # Apply sliding windows to the testing files
    test_windows, test_labels = load_and_apply_sliding_windows2(test_files, window_size, step_size, label)

    # Return the sliding windows and their labels for both training and testing sets
    return train_windows, train_labels, test_windows, test_labels

In [ ]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data2 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data2[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data2[activity]['train_windows'], train_test_data2[activity]['train_labels'],
     train_test_data2[activity]['test_windows'], train_test_data2[activity]['test_labels']) = process_activity2(
        activity, label, your_dataset_path, window_size=50, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.

KeyboardInterrupt: 

In [ ]:
# Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train2, y_train2 = combine_data(train_test_data2, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test2, y_test2 = combine_data(train_test_data2, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.

In [ ]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train2 shape: {X_train2.shape}, y_train2 shape: {y_train2.shape}")
print(f"X_test2 shape: {X_test2.shape}, y_test2 shape: {y_test2.shape}")

X_train2 shape: (57159, 50, 21), y_train2 shape: (57159,)
X_test2 shape: (14333, 50, 21), y_test2 shape: (14333,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [ ]:
# Define the model architecture
model3 = Sequential()

# Convolutional layers
model3.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(50, 21)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.3))

model3.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.3))

# LSTM layers for temporal dependencies
model3.add(LSTM(100, return_sequences=True))
model3.add(LSTM(50))

# Dense layers for classification
model3.add(Dense(50, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(4, activation='softmax'))  # 4 classes

# Compile the model
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for improved performance
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
]

# Train the model
history = model3.fit(
    X_train2, y_train2,
    epochs=50,
    batch_size=32,
    validation_data=(X_test2, y_test2),
    callbacks=callbacks
)

# Evaluate the model on test data
test_loss, test_accuracy = model3.evaluate(X_test2, y_test2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict and print class-wise accuracy
y_pred = model3.predict(X_test2)
y_pred_classes = y_pred.argmax(axis=1)

# Classification report
report = classification_report(y_test2, y_pred_classes, target_names=['breathingNormally', 'coughing', 'hyperventilation', 'other'])
print(report)

C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 38s 19ms/step - accuracy: 0.5388 - loss: 1.0740 - val_accuracy: 0.5946 - val_loss: 0.9328 - learning_rate: 0.0010
Epoch 2/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 35s 19ms/step - accuracy: 0.5881 - loss: 0.8911 - val_accuracy: 0.6034 - val_loss: 0.8747 - learning_rate: 0.0010
Epoch 3/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.5973 - loss: 0.8633 - val_accuracy: 0.6152 - val_loss: 0.8385 - learning_rate: 0.0010
Epoch 4/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 35s 20ms/step - accuracy: 0.6060 - loss: 0.8413 - val_accuracy: 0.6016 - val_loss: 0.8969 - learning_rate: 0.0010
Epoch 5/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.6175 - loss: 0.8296 - val_accuracy: 0.6089 - val_loss: 0.8707 - learning_rate: 0.0010
Epoch 6/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.6226 - loss: 0.8166 - val_accuracy: 0.6228 - val_loss: 0.8487 - learning_rate: 0.0010
Epoch 7/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 40s 22ms/step - accura

using previous best model (not GPT's):

In [ ]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot2 = encoder.fit_transform(y_train2.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot2 = encoder.transform(y_test2.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [ ]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train2 shape: {X_train2.shape}, y_train2 shape: {y_train2.shape}")
print(f"X_test2 shape: {X_test2.shape}, y_test2 shape: {y_test2.shape}")

# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot2 shape: {y_train_one_hot2.shape}, y_test_one_hot2 shape: {y_test_one_hot2.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

X_train2 shape: (57159, 50, 21), y_train2 shape: (57159,)
X_test2 shape: (14333, 50, 21), y_test2 shape: (14333,)
y_train_one_hot2 shape: (57159, 4), y_test_one_hot2 shape: (14333, 4)


In [ ]:
# Train the model
best_model2 = build_best_tuning_model(input_shape=(50,21))
history = best_model2.fit(
    X_train2, y_train_one_hot2,
    epochs=50,
    batch_size=32,
    validation_data=(X_test2, y_test_one_hot2),
    callbacks=callbacks
)

In [ ]:


# Evaluate the model on test data
test_loss, test_accuracy = best_model2.evaluate(X_test2, y_test_one_hot2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict and print class-wise accuracy
y_pred = best_model2.predict(X_test2)
y_pred_classes = y_pred.argmax(axis=1)

# Classification report
report = classification_report(y_test2, y_pred_classes, target_names=['breathingNormally', 'coughing', 'hyperventilation', 'other'])
print(report)

448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4411 - loss: 1.1358
Test Accuracy: 59.18%
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
                   precision    recall  f1-score   support

breathingNormally       0.70      0.30      0.42      2793
         coughing       0.55      0.66      0.60      2169
 hyperventilation       0.52      0.21      0.29      2167
            other       0.60      0.80      0.68      7204

         accuracy                           0.59     14333
        macro avg       0.59      0.49      0.50     14333
     weighted avg       0.60      0.59      0.56     14333



### Transformer model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model

In [ ]:
def build_transformer_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Multi-Head Attention Layer
    x = MultiHeadAttention(num_heads=4, key_dim=64)(inputs, inputs)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dropout(0.3)(x)

    # Additional Feedforward Layer with residual connection
    x = Dense(128, activation='relu')(x)
    x = GlobalAveragePooling1D()(x)

    # Final dense layers for classification
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# Instantiate and train the Transformer model
transformer_model = build_transformer_model(input_shape=input_shape, num_classes=4)
history = transformer_model.fit(
    X_train, y_train,  # Note: y_train2 should be integer encoded, not one-hot for 'sparse_categorical_crossentropy'
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
)

# Evaluate the model
test_loss, test_accuracy = transformer_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.4953 - loss: 1.2526 - val_accuracy: 0.5026 - val_loss: 1.2360 - learning_rate: 5.0000e-04
Epoch 2/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.5051 - loss: 1.2365 - val_accuracy: 0.5026 - val_loss: 1.2312 - learning_rate: 5.0000e-04
Epoch 3/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.5098 - loss: 1.2269 - val_accuracy: 0.5339 - val_loss: 1.1849 - learning_rate: 5.0000e-04
Epoch 4/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.5317 - loss: 1.1937 - val_accuracy: 0.5332 - val_loss: 1.1791 - learning_rate: 5.0000e-04
Epoch 5/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.5309 - loss: 1.1840 - val_accuracy: 0.5383 - val_loss: 1.1481 - learning_rate: 5.0000e-04
Epoch 6/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.5396 - loss: 1.1390 - val_accuracy: 0.5769 - val_loss: 1.0447 - learning_rate: 5.0000e-04
Epoch 7/50
1787/1787 ━━━━━━━━━━━━━━━━━━━━ 

# Models from papers

https://ieeexplore.ieee.org/abstract/document/10331170

gave up on this because their were layer shape mismatches in the model and it wasn't immediately obvious from the paper how they resolved those

also haven't implemented their input optimisation

In [ ]:
def build_1d_cnn_model4(hp):
    """
    Builds and compiles a 1D CNN model for multi-class classification.

    Args:
        input_shape (tuple): The shape of the input data (timesteps, features).
        num_classes (int): The number of output classes.

    Returns:
        model (Sequential): Compiled 1D CNN model.
    """
    model = Sequential()

    # First Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape))

    #model.add(MaxPooling1D(pool_size=2))

    # Second Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128,
                     kernel_size=5,
                     activation='relu',
                     kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3])),
                     bias_regularizer=l2(hp.Choice('l2_regularization_bias', values=[1e-4, 1e-3]))
                    ))

    #model.add(MaxPooling1D(pool_size=2))

    # Flatten the output from the convolutional layers

    #model.add(Flatten())

    # Fully connected layer

    #model.add(Dense(128, activation='relu'))

    # Dropout layer for regularization
    # You can try experimenting with different dropout rates
    model.add(Dropout(0.3))

    model.add(MaxPooling1D(pool_size=2))

    model.add(Dense(100,
                    activation='softmax',
                     kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3])),
                     bias_regularizer=l2(hp.Choice('l2_regularization_bias', values=[1e-4, 1e-3]))
                    ))

    # Output layer with softmax for multi-class classification
    model.add(Dense(4,
                    activation='softmax',
                     kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3])),
                     bias_regularizer=l2(hp.Choice('l2_regularization_bias', values=[1e-4, 1e-3]))
                    ))

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    #  Prints a detailed summary of the model, showing the layers, their output shapes, and the number of trainable parameters
    model.summary()

    return model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.regularizers import l2
import keras_tuner as kt

In [ ]:
tuner4 = kt.RandomSearch(
    build_1d_cnn_model4,
    objective='val_accuracy',  # Tuning for validation accuracy
    max_trials=20,             # Number of different hyperparameter sets to try
    executions_per_trial=2,    # Average results over multiple runs to reduce variance
    directory='my_tuning_dir', # Directory to save tuning results
    project_name='respiratory_activity_classification_model4'
)

# Split your data if not already done
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

tuner4.search(X_train, y_train_one_hot,
             epochs=20,               # Number of epochs for each trial
             validation_data=(X_test, y_test_one_hot),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

model4 = tuner4.get_best_models(num_models=1)[0]

Reloading Tuner from my_tuning_dir\respiratory_activity_classification_model4\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.001             |0.001             |l2_regularization
0.001             |0.001             |l2_regularization_bias



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 46, 128)        │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 42, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 42, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 21, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 21, 100)        │        12,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 21, 4)          │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,400 (380.47 KB)

 Trainable params: 97,400 (380.47 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


Traceback (most recent call last):
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\Py

window_size = 4s = 100 timesteps (untested)

In [ ]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data3 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data3[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data3[activity]['train_windows'], train_test_data3[activity]['train_labels'],
     train_test_data3[activity]['test_windows'], train_test_data3[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=100, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.

In [ ]:
# Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train3, y_train3 = combine_data(train_test_data3, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test3, y_test3 = combine_data(train_test_data3, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.

In [ ]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot3 = encoder.fit_transform(y_train3.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot3 = encoder.transform(y_test3.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [ ]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train3 shape: {X_train3.shape}, y_train3 shape: {y_train3.shape}")
print(f"X_test3 shape: {X_test3.shape}, y_test3 shape: {y_test3.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot3 shape: {y_train_one_hot3.shape}, y_test_one_hot3 shape: {y_test_one_hot3.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

## s-GAN
https://project-archive.inf.ed.ac.uk/ug4/20212442/ug4_proj.pdf#page=26.16

### data processing, window size 128 overlapping

In [30]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data4 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data4[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data4[activity]['train_windows'], train_test_data4[activity]['train_labels'],
     train_test_data4[activity]['test_windows'], train_test_data4[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=128, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train4, y_train4 = combine_data(train_test_data4, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test4, y_test4 = combine_data(train_test_data4, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder4 = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot4 = encoder4.fit_transform(y_train4.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot4 = encoder4.transform(y_test4.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train4 shape: {X_train4.shape}, y_train4 shape: {y_train4.shape}")
print(f"X_test4 shape: {X_test4.shape}, y_test4 shape: {y_test4.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot4 shape: {y_train_one_hot4.shape}, y_test_one_hot4 shape: {y_test_one_hot4.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape4 = (X_train4.shape[1], X_train4.shape[2])

X_train4 shape: (49890, 128, 3), y_train4 shape: (49890,)
X_test4 shape: (12508, 128, 3), y_test4 shape: (12508,)
y_train_one_hot4 shape: (49890, 4), y_test_one_hot4 shape: (12508, 4)


### model, window size 128

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim=100, output_channels=3):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.fc1 = nn.Linear(latent_dim, 1024 * 4)  # Output: (batch_size, 4096)

        self.deconv1 = nn.Sequential(
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 8)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 256, 16)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 128, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 64, 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose1d(64, output_channels, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 3, 128)
            nn.Tanh()
        )

    def forward(self, z):
        x = self.fc1(z)
        x = x.view(-1, 1024, 4)
        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.deconv4(x)
        x = self.deconv5(x)
        return x  # Output shape: (batch_size, 3, 128)

class Discriminator(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 1024, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )

        self.shared_features = nn.Flatten()  # Flattens to (batch_size, 1024*32)

        self.classifier = nn.Linear(1024 * 32, num_classes)
        self.discriminator = nn.Sequential(
            nn.Linear(1024 * 32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        features = self.shared_features(x)

        class_output = self.classifier(features)
        real_fake_output = self.discriminator(features)
        return class_output, real_fake_output


In [ ]:
def train_sgan(generator, discriminator, dataloader, num_classes, num_epochs=30, latent_dim=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)

    criterion_class = nn.CrossEntropyLoss()
    criterion_real_fake = nn.BCELoss()

    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (x_real, labels_real) in enumerate(dataloader):
            x_real = x_real.to(device)  # Shape: (batch_size, 3, 128)
            labels_real = labels_real.to(device)
            batch_size = x_real.size(0)
            half_batch = batch_size // 2

            # Prepare labels
            real_labels = torch.ones(half_batch, 1).to(device)
            fake_labels = torch.zeros(half_batch, 1).to(device)

            # ====================
            # Train Discriminator
            # ====================
            discriminator.train()
            optimizer_D.zero_grad()

            # Train on real data
            x_real_half = x_real[:half_batch]
            labels_real_half = labels_real[:half_batch]
            class_output_real, real_fake_output_real = discriminator(x_real_half)
            d_loss_real_class = criterion_class(class_output_real, labels_real_half)
            d_loss_real_rf = criterion_real_fake(real_fake_output_real, real_labels)

            # Train on fake data
            z = torch.randn(half_batch, latent_dim).to(device)
            x_fake = generator(z)
            class_output_fake, real_fake_output_fake = discriminator(x_fake.detach())
            d_loss_fake_rf = criterion_real_fake(real_fake_output_fake, fake_labels)

            # Total discriminator loss
            d_loss = d_loss_real_class + d_loss_real_rf + d_loss_fake_rf
            d_loss.backward()
            optimizer_D.step()

            # ====================
            # Train Generator
            # ====================
            # Freeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = False

            generator.train()
            optimizer_G.zero_grad()

            # Generate fake data
            z = torch.randn(batch_size, latent_dim).to(device)
            x_fake = generator(z)

            # Forward pass through discriminator
            _, real_fake_output_fake = discriminator(x_fake)

            # Generator wants discriminator to classify fake data as real
            real_labels_g = torch.ones(batch_size, 1).to(device)
            g_loss_rf = criterion_real_fake(real_fake_output_fake, real_labels_g)
            g_loss = g_loss_rf
            g_loss.backward()
            optimizer_G.step()

            # Unfreeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = True

            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')


In [ ]:
import numpy as np

# Compute the mean and standard deviation from the training data
mean = X_train4.mean()
std = X_train4.std()

# Normalize the training and test data using the training mean and std
X_train4 = (X_train4 - mean) / std
X_test4 = (X_test4 - mean) / std

# Clip the data to the range [-1, 1]
X_train4 = np.clip(X_train4, -1, 1)
X_test4 = np.clip(X_test4, -1, 1)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X  # NumPy array of shape (num_samples, seq_length, num_features)
        self.y = y  # NumPy array of shape (num_samples,)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Get the sample and label
        x = self.X[idx]  # Shape: (128, 3)
        y = self.y[idx]

        # Convert to torch tensors and transpose x to shape (num_features, seq_length)
        x = torch.tensor(x, dtype=torch.float32).transpose(0, 1)  # Shape: (3, 128)
        y = torch.tensor(y, dtype=torch.long)

        return x, y

In [ ]:
batch_size = 128  # Adjust based on your memory constraints

# Create dataset and dataloader
train_dataset = CustomDataset(X_train4, y_train4)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create dataset and dataloader
train_dataset = CustomDataset(X_train4, y_train4)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Example usage
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim = 100
    num_classes = 4  # Replace with the actual number of activity classes
    num_epochs = 30
    batch_size = 128

    # Initialize models
    generator = Generator(latent_dim=latent_dim)
    discriminator = Discriminator(num_classes=num_classes)

    # Create DataLoader
    train_dataset = CustomDataset(X_train4, y_train4)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Train the S-GAN
    train_sgan(generator, discriminator, train_loader, num_classes, num_epochs, latent_dim)

Epoch [1/30] Batch [0/390] D_loss: 2.7580, G_loss: 0.6114
Epoch [1/30] Batch [100/390] D_loss: 1.6198, G_loss: 2.4529
Epoch [1/30] Batch [200/390] D_loss: 1.5820, G_loss: 2.6505
Epoch [1/30] Batch [300/390] D_loss: 2.1992, G_loss: 2.6483
Epoch [2/30] Batch [0/390] D_loss: 1.8996, G_loss: 2.0298
Epoch [2/30] Batch [100/390] D_loss: 2.0109, G_loss: 1.6630
Epoch [2/30] Batch [200/390] D_loss: 1.7571, G_loss: 1.6895
Epoch [2/30] Batch [300/390] D_loss: 2.0927, G_loss: 1.8195
Epoch [3/30] Batch [0/390] D_loss: 1.8469, G_loss: 1.8593
Epoch [3/30] Batch [100/390] D_loss: 1.9107, G_loss: 2.0856
Epoch [3/30] Batch [200/390] D_loss: 1.8482, G_loss: 1.3632
Epoch [3/30] Batch [300/390] D_loss: 1.9381, G_loss: 1.8825
Epoch [4/30] Batch [0/390] D_loss: 2.3155, G_loss: 1.9598
Epoch [4/30] Batch [100/390] D_loss: 2.0052, G_loss: 1.1003
Epoch [4/30] Batch [200/390] D_loss: 2.1023, G_loss: 1.3040
Epoch [4/30] Batch [300/390] D_loss: 2.0882, G_loss: 1.0938
Epoch [5/30] Batch [0/390] D_loss: 2.1074, G_los

In [ ]:
# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset = CustomDataset(X_test4, y_test4)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Set the discriminator to evaluation mode
discriminator.eval()


Discriminator(
  (conv1): Sequential(
    (0): Conv1d(3, 512, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv1d(512, 1024, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (shared_features): Flatten(start_dim=1, end_dim=-1)
  (classifier): Linear(in_features=32768, out_features=4, bias=True)
  (discriminator): Sequential(
    (0): Linear(in_features=32768, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Lists to store predictions and true labels
all_preds = []
all_labels = []
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for x_test, y_test in test_loader:
        x_test = x_test.to(device)
        y_test = y_test.to(device)

        # Get the class outputs from the discriminator
        class_output, _ = discriminator(x_test)

        # Get the predicted class (the index with the highest score)
        _, preds = torch.max(class_output, 1)

        # Store predictions and true labels
        all_preds.append(preds.cpu().numpy())
        all_labels.append(y_test.cpu().numpy())

# Concatenate all predictions and labels into single arrays
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

# Generate a classification report
report = classification_report(all_labels, all_preds, digits=4)
print("Classification Report:")
print(report)

# Generate a confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)


Accuracy: 0.5625
Classification Report:
              precision    recall  f1-score   support

           0     0.8605    0.1364    0.2355      2441
           1     0.5393    0.3093    0.3931      1885
           2     0.6264    0.0904    0.1580      1891
           3     0.5526    0.9455    0.6975      6294

    accuracy                         0.5625     12511
   macro avg     0.6447    0.3704    0.3710     12511
weighted avg     0.6218    0.5625    0.4800     12511

Confusion Matrix:
[[ 333   76   19 2013]
 [  35  583   45 1222]
 [   2  134  171 1584]
 [  17  288   38 5951]]


## AC-GAN

### train

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [46]:
class Generator6(nn.Module):
  def __init__(self, latent_dim=100, num_classes=4, output_channels=3):
    super(Generator6, self).__init__()
    self.latent_dim = latent_dim
    self.num_classes = num_classes

    # Layers for processing noise vector z
    self.fc_z = nn.Linear(latent_dim, 1024 * 4)  # Output: (batch_size, 4096)

    # Layers for processing class labels c
    self.fc_c = nn.Linear(num_classes, 1024 * 4)  # Output: (batch_size, 4096)

    self.deconv1 = nn.Sequential(
      nn.ConvTranspose1d(2048, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 8)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv2 = nn.Sequential(
      nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 256, 16)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv3 = nn.Sequential(
      nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 128, 32)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv4 = nn.Sequential(
      nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 64, 64)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv5 = nn.Sequential(
      nn.ConvTranspose1d(64, output_channels, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 3, 128)
      nn.Tanh()
    )

  def forward(self, z, c):
    # Process noise vector z
    x_z = self.fc_z(z)  # Shape: (batch_size, 4096)
    x_z = x_z.view(-1, 1024, 4)

    # One-hot encode class labels and process
    c = F.one_hot(c, num_classes=self.num_classes).float()
    x_c = self.fc_c(c)  # Shape: (batch_size, 4096)
    x_c = x_c.view(-1, 1024, 4)

    # Concatenate feature maps
    x = torch.cat([x_z, x_c], dim=1)  # Shape: (batch_size, 2048, 4)

    # Pass through deconvolutional layers
    x = self.deconv1(x)
    x = self.deconv2(x)
    x = self.deconv3(x)
    x = self.deconv4(x)
    x = self.deconv5(x)
    return x  # Output shape: (batch_size, 3, 128)


In [47]:
class Discriminator6(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator6, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 1024, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )

        self.shared_features = nn.Flatten()  # Flattens to (batch_size, 1024*32)

        self.classifier = nn.Linear(1024 * 32, num_classes)
        self.discriminator = nn.Sequential(
            nn.Linear(1024 * 32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        features = self.shared_features(x)

        class_output = self.classifier(features)
        real_fake_output = self.discriminator(features)
        return class_output, real_fake_output

In [48]:
import numpy as np
from torch.utils.data import Dataset, DataLoader


def train_acgan6(generator, discriminator, dataloader, num_classes, num_epochs=30, latent_dim=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)

    criterion_class = nn.CrossEntropyLoss()
    criterion_real_fake = nn.BCELoss()

    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (x_real, labels_real) in enumerate(dataloader):
            x_real = x_real.to(device)
            labels_real = labels_real.to(device)
            batch_size = x_real.size(0)
            half_batch = batch_size // 2

            # Prepare labels
            real_labels = torch.ones(half_batch, 1).to(device)
            fake_labels = torch.zeros(half_batch, 1).to(device)

            # ====================
            # Train Discriminator
            # ====================
            discriminator.train()
            optimizer_D.zero_grad()

            # Train on real data
            x_real_half = x_real[:half_batch]
            labels_real_half = labels_real[:half_batch]
            class_output_real, real_fake_output_real = discriminator(x_real_half)
            d_loss_real_class = criterion_class(class_output_real, labels_real_half)
            d_loss_real_rf = criterion_real_fake(real_fake_output_real, real_labels)

            # Train on fake data
            z = torch.randn(half_batch, latent_dim).to(device)
            c_fake = torch.randint(0, num_classes, (half_batch,)).to(device)
            x_fake = generator(z, c_fake)
            class_output_fake, real_fake_output_fake = discriminator(x_fake.detach())
            d_loss_fake_class = criterion_class(class_output_fake, c_fake)
            d_loss_fake_rf = criterion_real_fake(real_fake_output_fake, fake_labels)

            # Total discriminator loss
            d_loss = d_loss_real_class + d_loss_real_rf + d_loss_fake_class + d_loss_fake_rf
            d_loss.backward()
            optimizer_D.step()

            # ====================
            # Train Generator
            # ====================
            # Freeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = False

            generator.train()
            optimizer_G.zero_grad()

            # Generate fake data
            z = torch.randn(batch_size, latent_dim).to(device)
            c_fake = torch.randint(0, num_classes, (batch_size,)).to(device)
            x_fake = generator(z, c_fake)

            # Forward pass through discriminator
            class_output_fake, real_fake_output_fake = discriminator(x_fake)

            # Generator wants discriminator to classify fake data as real and with correct class
            real_labels_g = torch.ones(batch_size, 1).to(device)
            g_loss_rf = criterion_real_fake(real_fake_output_fake, real_labels_g)
            g_loss_class = criterion_class(class_output_fake, c_fake)
            g_loss = g_loss_rf + g_loss_class
            g_loss.backward()
            optimizer_G.step()

            # Unfreeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = True

            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')

# Normalize the data as before
mean6 = X_train4.mean()
std6 = X_train4.std()

X_train4 = (X_train4 - mean6) / std6
X_test4 = (X_test4 - mean6) / std6

X_train4 = np.clip(X_train4, -1, 1)
X_test4 = np.clip(X_test4, -1, 1)

class CustomDataset6(Dataset):
    def __init__(self, X, y):
        self.X = X  # NumPy array of shape (num_samples, seq_length, num_features)
        self.y = y  # NumPy array of shape (num_samples,)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Get the sample and label
        x = self.X[idx]  # Shape: (128, 3)
        y = self.y[idx]

        # Convert to torch tensors and transpose x to shape (num_features, seq_length)
        x = torch.tensor(x, dtype=torch.float32).transpose(0, 1)  # Shape: (3, 128)
        y = torch.tensor(y, dtype=torch.long)

        return x, y

In [55]:
# Example usage
from torch.utils.data import DataLoader, RandomSampler
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim6 = 100
    num_classes6 = 4  # Replace with the actual number of activity classes

    #num_epochs6 = 30
    batch_size6 = 128

    # Initialize models
    generator6 = Generator6(latent_dim=latent_dim6, num_classes=num_classes6)
    discriminator6 = Discriminator6(num_classes=num_classes6)

    # Create DataLoader
    train_dataset6 = CustomDataset6(X_train4, y_train4)
    sampler6 = RandomSampler(train_dataset6, replacement=True)
    train_loader6 = DataLoader(train_dataset6, batch_size=batch_size6, sampler=sampler6)

    num_mini_batches_per_epoch = len(train_loader6)
    num_epochs6 = (25000 + num_mini_batches_per_epoch - 1) // num_mini_batches_per_epoch  # Ceiling division

    # Train the AC-GAN
    train_acgan6(generator6, discriminator6, train_loader6, num_classes6, num_epochs6, latent_dim6)


Epoch [1/65] Batch [0/390] D_loss: 4.1760, G_loss: 2.0257
Epoch [1/65] Batch [100/390] D_loss: 2.2235, G_loss: 2.3036
Epoch [1/65] Batch [200/390] D_loss: 1.8291, G_loss: 3.6332
Epoch [1/65] Batch [300/390] D_loss: 2.0946, G_loss: 2.4428
Epoch [2/65] Batch [0/390] D_loss: 2.1174, G_loss: 2.3532
Epoch [2/65] Batch [100/390] D_loss: 1.6486, G_loss: 2.5491
Epoch [2/65] Batch [200/390] D_loss: 1.8233, G_loss: 2.3020
Epoch [2/65] Batch [300/390] D_loss: 2.0460, G_loss: 2.0406
Epoch [3/65] Batch [0/390] D_loss: 2.2771, G_loss: 2.2608
Epoch [3/65] Batch [100/390] D_loss: 2.1957, G_loss: 2.4005
Epoch [3/65] Batch [200/390] D_loss: 2.2194, G_loss: 2.0241
Epoch [3/65] Batch [300/390] D_loss: 2.3286, G_loss: 2.6675
Epoch [4/65] Batch [0/390] D_loss: 1.8337, G_loss: 2.2396
Epoch [4/65] Batch [100/390] D_loss: 1.8765, G_loss: 1.8440
Epoch [4/65] Batch [200/390] D_loss: 2.0048, G_loss: 1.7572
Epoch [4/65] Batch [300/390] D_loss: 1.8870, G_loss: 2.0093
Epoch [5/65] Batch [0/390] D_loss: 2.0214, G_los

### test

In [56]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset6 = CustomDataset6(X_test4, y_test4)

# Create DataLoader
test_loader6 = DataLoader(test_dataset6, batch_size=batch_size6, shuffle=False)

# Set the discriminator to evaluation mode
discriminator6.eval()

# Lists to store predictions and true labels
all_preds6 = []
all_labels6 = []
# Define the device
device6 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for x_test, y_test in test_loader6:
        x_test = x_test.to(device6)
        y_test = y_test.to(device6)

        # Get the class outputs from the discriminator
        class_output, _ = discriminator6(x_test)

        # Get the predicted class (the index with the highest score)
        _, preds = torch.max(class_output, 1)

        # Store predictions and true labels
        all_preds6.append(preds.cpu().numpy())
        all_labels6.append(y_test.cpu().numpy())

# Concatenate all predictions and labels into single arrays
all_preds6 = np.concatenate(all_preds6)
all_labels6 = np.concatenate(all_labels6)


# Compute accuracy
accuracy6 = accuracy_score(all_labels6, all_preds6)
print(f"Accuracy: {accuracy6:.4f}")

# Generate a classification report
report6 = classification_report(all_labels6, all_preds6, digits=4)
print("Classification Report:")
print(report6)

# Generate a confusion matrix
cm6 = confusion_matrix(all_labels6, all_preds6)
print("Confusion Matrix:")
print(cm6)


Accuracy: 0.5790
Classification Report:
              precision    recall  f1-score   support

           0     0.6274    0.3120    0.4168      2439
           1     0.5829    0.3607    0.4456      1891
           2     0.5240    0.1270    0.2044      1890
           3     0.5750    0.8841    0.6968      6288

    accuracy                         0.5790     12508
   macro avg     0.5773    0.4209    0.4409     12508
weighted avg     0.5787    0.5790    0.5298     12508

Confusion Matrix:
[[ 761   30   22 1626]
 [ 145  682   61 1003]
 [  46  125  240 1479]
 [ 261  333  135 5559]]


### data processing, window size 24 non-overlapping

In [57]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data24 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data24[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data24[activity]['train_windows'], train_test_data24[activity]['train_labels'],
     train_test_data24[activity]['test_windows'], train_test_data24[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=24, step_size=24)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train24, y_train24 = combine_data(train_test_data24, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test24, y_test24 = combine_data(train_test_data24, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder24 = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot24 = encoder24.fit_transform(y_train24.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot24 = encoder24.transform(y_test24.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train24 shape: {X_train24.shape}, y_train24 shape: {y_train24.shape}")
print(f"X_test24 shape: {X_test24.shape}, y_test24 shape: {y_test24.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot24 shape: {y_train_one_hot24.shape}, y_test_one_hot24 shape: {y_test_one_hot24.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape24 = (X_train24.shape[1], X_train24.shape[2])

X_train24 shape: (118850, 24, 3), y_train24 shape: (118850,)
X_test24 shape: (29815, 24, 3), y_test24 shape: (29815,)
y_train_one_hot24 shape: (118850, 4), y_test_one_hot24 shape: (29815, 4)


### train, window size 24 non-overlapping

### data processing, window size 24 overlapping

In [58]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data24o = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data24o[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data24o[activity]['train_windows'], train_test_data24o[activity]['train_labels'],
     train_test_data24o[activity]['test_windows'], train_test_data24o[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=24, step_size=12)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train24o, y_train24o = combine_data(train_test_data24o, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test24o, y_test24o = combine_data(train_test_data24o, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder24o = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot24o = encoder24o.fit_transform(y_train24o.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot24o = encoder24o.transform(y_test24o.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train24 shape: {X_train24o.shape}, y_train24 shape: {y_train24o.shape}")
print(f"X_test24 shape: {X_test24o.shape}, y_test24 shape: {y_test24o.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot24 shape: {y_train_one_hot24o.shape}, y_test_one_hot24 shape: {y_test_one_hot24o.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape24o = (X_train24o.shape[1], X_train24o.shape[2])

X_train24 shape: (236990, 24, 3), y_train24 shape: (236990,)
X_test24 shape: (59313, 24, 3), y_test24 shape: (59313,)
y_train_one_hot24 shape: (236990, 4), y_test_one_hot24 shape: (59313, 4)


### train, window size 24 overlapping